In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("chicago_data_assignment.db")
cur = con.cursor()

In [4]:
import pandas
census_df = pandas.read_csv("ChicagoCensusData.csv")
census_df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

In [5]:
crime_df = pandas.read_csv("ChicagoCrimeData.csv")
crime_df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False,method="multi")

In [6]:
public_schools_df = pandas.read_csv("ChicagoPublicSchools.csv")
public_schools_df.to_sql("CHICAGO_PUBLIC_SCHOOLS", con, if_exists='replace', index=False,method="multi")

C:\Users\amic7\anaconda3\lib\site-packages\pandas\core\generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [7]:
%sql sqlite:///chicago_data_assignment.db

Find the total number of crimes recorded in the CRIME table

In [8]:
%sql SELECT COUNT(*) FROM CHICAGO_CRIME_DATA 

 * sqlite:///chicago_data_assignment.db
Done.


COUNT(*)
533


List community areas with per capita income less than 11000

In [9]:
%sql SELECT COMMUNITY_AREA_NAME, PER_CAPITA_INCOME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000

 * sqlite:///chicago_data_assignment.db
Done.


COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [10]:
%sql SELECT CASE_NUMBER, DESCRIPTION FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%MINOR%'

 * sqlite:///chicago_data_assignment.db
Done.


CASE_NUMBER,DESCRIPTION
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


List all kidnapping crimes involving a child?

In [11]:
%sql SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION \
LIKE '%CHILD%'

 * sqlite:///chicago_data_assignment.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


What kinds of crimes were recorded at schools?

In [12]:
%sql SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION AS CRIME_DESCRIPTION FROM CHICAGO_CRIME_DATA \
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///chicago_data_assignment.db
Done.


CASE_NUMBER,PRIMARY_TYPE,CRIME_DESCRIPTION
HL353697,BATTERY,SIMPLE
HL725506,BATTERY,PRO EMP HANDS NO/MIN INJURY
HP716225,BATTERY,SIMPLE
HH639427,BATTERY,SIMPLE
JA460432,BATTERY,SIMPLE
HS200939,CRIMINAL DAMAGE,TO VEHICLE
HK577020,NARCOTICS,POSS: HEROIN(WHITE)
HS305355,NARCOTICS,MANU/DEL:CANNABIS 10GM OR LESS
HT315369,ASSAULT,PRO EMP HANDS NO/MIN INJURY
HR585012,CRIMINAL TRESPASS,TO LAND


List the average safety score for each type of school. NOTE: I NEED TO CHANGE THE COLUMN NAME WITH UNDERSCORES TO EXECUTE THIS QUERY.

In [13]:
%sql SELECT "Elementary, Middle, or High School", AVG(SAFETY_SCORE) \
FROM CHICAGO_PUBLIC_SCHOOLS \
GROUP BY "Elementary, Middle, or High School";


 * sqlite:///chicago_data_assignment.db
(sqlite3.OperationalError) near "or": syntax error
[SQL: SELECT Elementary, Middle, or High School , AVG(SAFETY_SCORE) FROM CHICAGO_PUBLIC_SCHOOLS GROUP BY Elementary, Middle, or High School ;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


List 5 community areas with highest % of households below poverty line

In [14]:
%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA \
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5


 * sqlite:///chicago_data_assignment.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


Which community area is most crime prone?

In [15]:
%sql SELECT COMMUNITY_AREA_NUMBER, COUNT(*) AS NUMBER_OF_CRIMES FROM CHICAGO_CRIME_DATA \
GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1

 * sqlite:///chicago_data_assignment.db
Done.


COMMUNITY_AREA_NUMBER,NUMBER_OF_CRIMES
25.0,43


Use a sub-query to find the name of the community area with highest hardship index

In [16]:
%sql SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, HARDSHIP_INDEX FROM CENSUS_DATA WHERE HARDSHIP_INDEX = \
(SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA)

 * sqlite:///chicago_data_assignment.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,HARDSHIP_INDEX
54.0,Riverdale,98.0


Use a sub-query to determine the Community Area Name with most number of crimes?

In [17]:
%sql SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE COMMUNITY_AREA_NUMBER = \
(SELECT COMMUNITY_AREA_NUMBER FROM CHICAGO_CRIME_DATA \
GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1)

 * sqlite:///chicago_data_assignment.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
25.0,Austin


In [18]:
con.close()
